In [1]:
import selenium
from selenium import webdriver
from selenium.webdriver import ActionChains

from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait

import time
import pandas as pd

In [ ]:
## 사용자가 조정해줘야 하는 값
## 추후 리스트 구조로 수정 가능

# URL : 주소 입력하기.
# num : 실행하는 번호 입력하기. (1부터 시작해서 계속 하나씩 늘려야함.)
# area : 지역 입력하기.
URL = 'https://map.naver.com/v5/search/%EA%B0%95%EB%8F%99%EA%B5%AC%20%EC%84%B1%EB%82%B4%EB%8F%99%20%EC%9D%8C%EC%8B%9D%EC%A0%90?c=14150737.8312652,4513303.6958368,15,0,0,0,dh'
num = 1
area = 'KongDong'

In [23]:
from selenium.webdriver.chrome.options import Options
chrome_options = Options()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')

driver = webdriver.Chrome(executable_path='/opt/ml/input/final/chromedriver', chrome_options=chrome_options)
driver.get(URL)

driver.switch_to.frame('searchIframe') #iframe 으로 이동하기 위한 코드
restaurants_list = pd.DataFrame()

In [25]:
from tqdm import tqdm
epoch = 6 # 6이 최대인듯
for _ in tqdm(range(epoch)):
    click_list = driver.find_elements(By.CLASS_NAME, "tzwk0")
    click_list[0].click() #아무거나 클릭을 해야 end키가 먹혀서 임의로 아무 클래스 클릭한거임
    for _ in range(10):
        driver.find_element(By.CSS_SELECTOR, "body").send_keys(Keys.END) # end키를 눌러서 아래까지 로딩
    time.sleep(2)
    click_list = driver.find_elements(By.CLASS_NAME, "tzwk0") # 식당 리스트
    name_list = driver.find_elements(By.CLASS_NAME, "TYaxT") # 식당 이름 리스트
    name_list = [user.text for user in name_list]
    tag_list = driver.find_elements(By.CLASS_NAME, "KCMnt") # 식당 종류 리스트
    tag_list = [user.text for user in tag_list]
    review_list = driver.find_elements(By.CLASS_NAME, "MVx6e") # 리뷰 개수 리스트
    review_list = [user.text.split("\n")[-1] for user in review_list]
    url_list = []
    for restaurant_link in click_list:
        restaurant_link.click()
        url_list.append(driver.current_url)
        time.sleep(0.5)
    restaurants_list2 = pd.DataFrame({'restaurant' : name_list, 'tag' : tag_list, 'url' : url_list, 'review' : review_list}, dtype = str)
    restaurants_list = pd.concat([restaurants_list, restaurants_list2], axis = 0, sort=False)
    driver.find_elements(By.CLASS_NAME, "yUtES")[1].click() #다음페이지 클릭
    time.sleep(2)

100%|██████████| 6/6 [04:03<00:00, 40.66s/it]


In [17]:
restaurants_list.to_csv(f'./area_csv/{area}/rest_{num}.csv', index=False)

,restaurant,tag,url,review
0,미테이블 강동직영점,양식,https://map.naver.com/v5/search/%EA%B0%95%EB%8...,영업 중리뷰 820길동 미테이블 유명 쉐프들이 만들어주는 요리
1,멘야세븐,일본식라면,https://map.naver.com/v5/search/%EA%B0%95%EB%8...,4.46리뷰 999+TV수요미식회
2,영화정해장국,해장국,https://map.naver.com/v5/search/%EA%B0%95%EB%8...,4.39리뷰 365
3,라라브레드 길동점,베이커리,https://map.naver.com/v5/search/%EA%B0%95%EB%8...,4.44리뷰 999+
4,어흥,요리주점,https://map.naver.com/v5/search/%EA%B0%95%EB%8...,4.89리뷰 562
5,소문난칼국수,"칼국수,만두",https://map.naver.com/v5/search/%EA%B0%95%EB%8...,4.46리뷰 999+
6,한주,요리주점,https://map.naver.com/v5/search/%EA%B0%95%EB%8...,4.78리뷰 268
7,제주은희네해장국 강동길동점,해장국,https://map.naver.com/v5/search/%EA%B0%95%EB%8...,영업 중리뷰 386서울 강동구 길동
8,비타밀,"다이어트,샐러드",https://map.naver.com/v5/search/%EA%B0%95%EB%8...,영업 중리뷰 52서울 강동구 길동
9,으뜸횟집,생선회,https://map.naver.com/v5/search/%EA%B0%95%EB%8...,4.34리뷰 673
